# 0. 초기 세팅

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Pytorch에서 gpu를 사용하는 방법.
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
# print('Current cuda device ', torch.cuda.current_device())

In [ ]:
import PIL

In [ ]:
from matplotlib import pyplot as plt
import tifffile as tiff
from PIL import Image
import random

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.metrics import MeanIoU

In [ ]:
from keras.utils import normalize
from keras.metrics import MeanIoU

In [ ]:
from keras import backend as K

In [ ]:
import tensorflow as tf
import segmentation_models as sm
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import keras

from keras.utils import normalize
from keras.metrics import MeanIoU

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [ ]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.backends.cudnn.enabled)
print(torch.backends.cudnn.version())

# 1-1. 이미지 분석

In [ ]:
from PIL import Image

In [ ]:
def show_hist(img):
    img_array = np.array(img)

    f = plt.figure(figsize=(18, 8))
    f.add_subplot(1, 2, 1)
    plt.imshow(img_array, cmap='gray')
    plt.title('image', fontsize=20)
    f.add_subplot(1, 2, 2)
    plt.hist(img_array.flatten(), bins = 256, alpha=0.5)
    plt.title('histogram', fontsize=20)
    plt.xlabel('gray levels', fontsize = 15)
    plt.ylabel('pixel count', fontsize = 15)
    plt.show()

In [ ]:
tmp_img_path = train_img_dir + data_train_img[0]
tmp_img = valid_images[5]

In [ ]:
show_hist(tmp_img)

In [ ]:
show_hist(tmp_img)

transform = A.Compose([A.Normalize()])
tmp_img2 = transform(image=tmp_img)['image']
show_hist(tmp_img2)

In [ ]:
from PIL import ImageOps

def img_preprocess(img):
    img = Image.fromarray(np.uint8(img))
    img4eq = ImageOps.equalize(img)
    img4eq_array = np.array(img4eq)
    return (img, img4eq, img4eq_array)

In [ ]:
def show_hist2(tmp_img, img4eq, img4_array, img4eq_array):
    f=plt.figure(figsize=(14, 10))
    f.add_subplot(2, 2, 1)
    plt.imshow(tmp_img, cmap='gray')
    plt.title('original Image', fontsize=20)
    f.add_subplot(2, 2, 2)
    plt.imshow(img4eq, cmap='gray')
    plt.title('equalized Image', fontsize=20)
    f.add_subplot(2, 2, 3)
    plt.hist(img4_array.flatten(), bins = 256, alpha=0.5)
    plt.title('histogram', fontsize=20)
    plt.xlabel('gray levels', fontsize = 15)
    plt.ylabel('pixel count', fontsize = 15)
    f.add_subplot(2, 2, 4)
    plt.hist(img4eq_array.flatten(), bins = 256, alpha=0.5)
    plt.title('histogram', fontsize=20)
    plt.xlabel('gray levels', fontsize = 15)
    plt.ylabel('pixel count', fontsize = 15)
    plt.show()

In [ ]:
result_contrast = img_preprocess(tmp_img)

In [ ]:
show_hist2(tmp_img, result_contrast[1], tmp_img, result_contrast[2])

In [ ]:
import PIL, PIL.ImageOps

In [ ]:
img = Image.fromarray(np.uint8(tmp_img))
result_img = PIL.ImageOps.autocontrast(img, cutoff=1)

In [ ]:
show_hist(result_img)

In [ ]:
denoised_img = cv2.fastNlMeansDenoisingColored(np.array(result_img), None, 5, 5, 5, 5)

In [ ]:
show_hist(denoised_img)

In [ ]:
img2 = Image.fromarray(np.uint8(denoised_img))
Sharpness_image = PIL.ImageEnhance.Sharpness(img2).enhance(1.2)

In [ ]:
show_hist(Sharpness_image)

In [ ]:
type(Sharpness_image)

In [ ]:
def img_preprocess(img):
    img = Image.fromarray(np.uint8(img))
    result_img = PIL.ImageOps.autocontrast(img, cutoff=1)
    denoised_img = cv2.fastNlMeansDenoisingColored(np.array(result_img), None, 5, 5, 5, 5)
    img2 = Image.fromarray(np.uint8(denoised_img))
    Sharpness_image = PIL.ImageEnhance.Sharpness(img2).enhance(1.2)
    return np.array(Sharpness_image)

### Custom Dataset & DataLoader

In [ ]:
class SatelliteDataset(Dataset):
    def __init__(self, dataset, lst_path, transform=None, infer=False):
        self.data = dataset
        self.transform = transform
        self.infer = infer
        self.lst_path = lst_path

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.lst_path[0] + self.data.iloc[idx, 0]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = img_preprocess(image)
                image = self.transform(image=image)['image']
            return image

        mask_path = self.lst_path[1] + self.data.iloc[idx, 1]
        mask = cv2.imread(mask_path, 0)

        if self.transform:
            image = img_preprocess(image)
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

In [ ]:
transform = A.Compose([
    A.Normalize(),        # 기존
    ToTensorV2()          # 기존
])

In [ ]:
trainset = SatelliteDataset(dataset = df_train, transform=transform, lst_path = [train_img_dir, train_mask_dir])
validset = SatelliteDataset(dataset = df_valid, transform=transform, lst_path = [valid_img_dir, valid_mask_dir])

In [ ]:
train_dataloader = DataLoader(trainset, batch_size=16, shuffle=True, num_workers=48)
valid_dataloader = DataLoader(validset, batch_size=16, shuffle=False, num_workers=48)

In [ ]:
valid_dataloader_2 = DataLoader(validset, batch_size=32, shuffle=False, num_workers=48)

# 성능 평가

**성능 평가를 위해 valid set에 대해서 결과 확인**

### 1) true_mask vs pred_mask 이미지 비교

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Union
from joblib import Parallel, delayed

In [ ]:
import matplotlib.pyplot as plt

def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        if i == 0:
            img = cv2.imread(display_list[i])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.astype(np.uint8).copy()
        if i == 1:
            img = cv2.imread(display_list[i], 0)
        if i == 2:
#             img = rle_decode(display_list[i], shape = (224, 224)) # shape 설정
            img = display_list[i]

        plt.imshow(img)
        plt.axis('off')
    plt.show()

idx = 1000

valid_img_path_idx = valid_img_dir + df_valid['img_path'][idx]
valid_mask_path_idx = valid_mask_dir + df_valid['mask_path'][idx]
valid_pred_mask_idx = tmp_pred4[idx]

display_list = [valid_img_path_idx, valid_mask_path_idx, valid_pred_mask_idx]
display(display_list)

In [ ]:
weighted_preds_test = np.where(weighted_preds > 0.5, 1, 0)

In [ ]:
idx = 5

valid_img_path_idx = valid_img_dir + df_valid['img_path'][idx]
valid_mask_path_idx = valid_mask_dir + df_valid['mask_path'][idx]

In [ ]:
display_list = [valid_img_path_idx, valid_mask_path_idx, tmp_pred4[idx]]
display(display_list)

### 2) Dice Score

In [ ]:
def dice_score(prediction: np.array, ground_truth: np.array, smooth=1e-7) -> float:
    '''
    Calculate Dice Score between two binary masks.
    '''
    intersection = np.sum(prediction * ground_truth)
    return (2.0 * intersection + smooth) / (np.sum(prediction) + np.sum(ground_truth) + smooth)


def calculate_dice_scores(pred_mask, gt_mask, img_shape=(224, 224)) -> List[float]:
    '''
    Calculate Dice scores for a dataset.
    '''

    def calculate_dice(pred_mask, gt_mask):
        pred_mask = pred_mask
        gt_mask = gt_mask


        if np.sum(gt_mask) > 0 or np.sum(pred_mask) > 0:
            return dice_score(pred_mask, gt_mask)
        else:
            return None  # No valid masks found, return None


    dice_scores = Parallel(n_jobs=-1)(
        delayed(calculate_dice)(pred, gt) for pred, gt in zip(pred_mask, gt_mask)
    )


    dice_scores = [score for score in dice_scores if score is not None]  # Exclude None values


    return np.mean(dice_scores)

In [ ]:
calculate_dice_scores(tmp_pred4, valid_masks, img_shape=(224, 224))

### 3) Class 별 IOU

In [ ]:
valid_masks.shape

In [ ]:
import numpy as np

def calculate_iou(outputs, labels, num_classes):
    iou_per_class = np.zeros(num_classes)

    for class_idx in range(num_classes):
        # 클래스별 True Positive, False Positive, False Negative 계산
        true_positive = np.sum(np.logical_and(outputs == class_idx, labels == class_idx))
        false_positive = np.sum(np.logical_and(outputs == class_idx, labels != class_idx))
        false_negative = np.sum(np.logical_and(outputs != class_idx, labels == class_idx))

        # 클래스별 IoU 계산
        iou = true_positive / (true_positive + false_positive + false_negative + 1e-10)
        iou_per_class[class_idx] = iou

    return iou_per_class

# 예시 사용
outputs = tmp_pred4
labels = valid_masks
num_classes = 2
iou_per_class = calculate_iou(outputs, labels, num_classes)

print("클래스별 IoU:", iou_per_class)

In [ ]:
totaliou = iou_per_class

In [ ]:
def printClassScores(totaliou):
    label = ['background', 'building']
    print('classes          IoU      nIoU')
    print('--------------------------------')
    for i, iou in enumerate(totaliou):
        labelName = label[i]
        iouStr = f'{iou:>5.3f}'
        niouStr = 'empty'
        print('{:<14}: '.format(labelName) + iouStr + '    ' + niouStr)
    print('--------------------------------')
    print(f'Score Average : {(np.sum(totaliou) / 2):>5.3f}' + '    ' + niouStr)

In [ ]:
printClassScores(totaliou)